In [ ]:

from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np


In [ ]:

input_size = 256
block_sizes = [  0 , 2 , 3 , 5 ]
output_dims = 8
image = np.asarray( Image.open( '366.jpg' ).resize( ( input_size , input_size ) ) )
image = np.expand_dims( image , 0 ) / 255
fig = plt.figure( figsize=( 15 , 20 ) )

def conv2D( x , units ):
    return tf.keras.layers.Conv2D( units , kernel_size=3 , kernel_initializer='he_uniform' , use_bias=False , activation='relu' , padding='same' )( x )

def read_image( path ):
    img = Image.open( path ).resize( ( input_size , input_size ) )
    return np.asarray( img )

def visualize_featuremap( feature_map ):
    feature_map -= feature_map.mean()
    feature_map /= feature_map.std()
    feature_map *=  64
    feature_map += 128
    feature_map = np.clip( feature_map , 0, 255 ).astype('uint8')
    return feature_map

init_conv_layer = tf.keras.layers.Conv2D( output_dims , kernel_size=3 , kernel_initializer='he_uniform' , use_bias=False , activation='relu' , padding='same' )

def make_model( block_size , skip_connections=True ):
    inputs = tf.keras.layers.Input( shape=( input_size , input_size , 3 ) )
    x = init_conv_layer( inputs )
    skip = x
    
    if skip_connections:
        for _ in range( block_size ):
            x = conv2D( x , output_dims )
            outputs = tf.keras.layers.Add()([ x , skip ])
    else:
        outputs = x
  
    model = tf.keras.models.Model( inputs , outputs )
    return model

i = 1
for size in block_sizes:
    model = make_model( size , size != 0 )
    feature_map = model( image )
    feature_map = feature_map.numpy()
    print( 'Mean {} and StdDev {} with block size {}'.format( feature_map.mean() , feature_map.std() , size ) )
    #print( 'Min {} and max {} with block size {}'.format( feature_map.min() , feature_map.max() , size ) )
    for j in range( output_dims ):
        fm_i = feature_map[ 0 , : , : , j ]
        fm_i = visualize_featuremap( fm_i )
        fig.add_subplot( len( block_sizes ) , output_dims , i )
        plt.imshow( fm_i )
        plt.title( 'Block size {}'.format( size ) )
        i += 1

plt.show()

